In [1]:
# Cufflinks wrapper on plotly
import cufflinks
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

from plotly.offline import iplot
cufflinks.go_offline()

# Set global theme


import plotly.graph_objects as go

import pandas as pd, numpy as np



## Data inputs

In [2]:
df_monthly =  pd.read_csv(r"https://raw.githubusercontent.com/Jan-Majewski/Medium_Articles/master/04_Plotly/monthly_rent.csv")

In [3]:
df_monthly

,year_month,total_monthly_price,offers,area,change
0,2020-07,2633.642857,6678,41.696361,0.000000
1,2020-08,2657.555923,5436,41.786738,0.907984
2,2020-09,2640.164394,6381,41.877247,-0.654418
3,2020-10,2606.674771,6005,41.612043,-1.268467
4,2020-11,2519.994276,8384,41.224971,-3.325328
5,2020-12,2516.056689,8238,41.338257,-0.156254
6,2021-01,2503.783532,8999,41.528673,-0.487793
7,2021-02,2559.851402,7740,42.085681,2.239326


In [4]:
df_monthly["change"] = df_monthly.total_monthly_price.pct_change()
df_monthly["change"] = df_monthly["change"].fillna(0)*100

# Line chart

## Basic line chart

In [5]:
trace0=go.Scatter(
            x=df_monthly.year_month,
            y=df_monthly.total_monthly_price,
            mode='lines',
            marker=dict(
            color="blue",
            size=5
            
            ),
        name="mean",

        )

data=[trace0]
    
figure=go.Figure(
    data=data,
    layout=go.Layout(
        title="Average monthly rent",
            yaxis=dict(title="Rent [PLN]"),
        xaxis=dict(title="Month",type="category")

    
    ))
iplot(figure)



## Line chart with annotation added

In [6]:
trace0=go.Scatter(
            x=df_monthly.year_month,
            y=df_monthly.total_monthly_price,
            mode='lines',
            marker=dict(
            color="blue",
            size=5,
          
            
            ),
        name="mean",

    #We can modify text and shown data in the text property
        text=['Mean monthly price: {} <br>monthly change {}% <br><br>Offers count: {}<br>Avg Area: {}'\
                              .format(df_monthly.total_monthly_price[i],
                                      df_monthly.change[i],
                                      df_monthly.offers[i],
                                      df_monthly.area[i],
                                     ) for i in range(0, df_monthly.shape[0])],

        )

data=[trace0]
    
figure=go.Figure(
    data=data,
    layout=go.Layout(
       title="Average monthly rent",
            yaxis=dict(title="Rent [PLN]"),
        xaxis=dict(title="Month",type="category")

    
    ))
iplot(figure)



## Improved annotation formatting

In [7]:
trace0=go.Scatter(
            x=df_monthly.year_month,
            y=df_monthly.total_monthly_price,
            hovertemplate= '%{text}',
            mode='lines',
            marker=dict(
            color="blue",

                
          
            
            ),
            name="",
    

    
        text=['<b>Mean monthly price: {:.0f} </b><br>monthly change {:.1f}% <br><br>Offers count: {:.0f}<br>Avg Area: {:.2f}'\
                              .format(df_monthly.total_monthly_price[i],
                                      df_monthly.change[i],
                                      df_monthly.offers[i],
                                      df_monthly.area[i],
                                     ) for i in range(0, df_monthly.shape[0])],

        )

data=[trace0]
    
figure=go.Figure(
    data=data,
    layout=go.Layout(
        title="<b>Average monthly rent",
            yaxis=dict(title="<b>Rent [PLN]"),
        xaxis=dict(title="<b>Month",type="category")

    
    ))
iplot(figure)


## Adding size and color to show 2 additional dimensions

In [8]:
custom_scale=[
    [0, 'rgb(192,0,0)'],
    [0.25,'rgb(255,0,0)'],
    [0.5,'rgb(255,255,0)'],
    [0.75,'rgb(146,208,80)'],
    [1,'rgb(84,130,53)'], 
             ]

In [9]:
trace0=go.Scatter(
            x=df_monthly.year_month,
            y=df_monthly.total_monthly_price,
            hovertemplate= '%{text}',
            
            mode='lines+markers',
            # Line style
            line=dict(color="grey", dash="dash"),
            # Marker style
            marker=dict(
                    color=df_monthly.change,
                    colorscale = custom_scale,
                    # Limiting color ranges for better readibility
                    cauto=False,
                    cmin=-1,
                    cmax=1,
                    # Setting size based on offers count
                    size=df_monthly.offers/500,opacity=1,

                    showscale=True,
                        #Adding title to color legend
                        colorbar=dict(title="<b>monthly change<br> [%] <br>")
                    ),
            name="",
        # Creating text for annotations
        text=['<b>Mean monthly price: {:.0f} </b><br>monthly change {:.1f}% <br><br>Offers count: {:.0f}<br>Avg Area: {:.2f}'\
                              .format(df_monthly.total_monthly_price[i],
                                      df_monthly.change[i],
                                      df_monthly.offers[i],
                                      df_monthly.area[i],
                                     ) for i in range(0, df_monthly.shape[0])],
        )


data=[trace0]
    
figure=go.Figure(
    data=data,
    layout=go.Layout(
        title="<b>Average monthly rent</b><br>line=rent, size=offers count, color=monthly change",
            yaxis=dict(title="<b>Rent [PLN]"),
        xaxis=dict(title="<b>Month",type="category")

    
    ))


iplot(figure)



## Advanced trend

In [10]:

df_monthly_dtr  =  pd.read_csv(r"https://raw.githubusercontent.com/Jan-Majewski/Medium_Articles/master/04_Plotly/monthly_rent_by_district.csv")

In [11]:
df_monthly_dtr

,year_month,district,total_monthly_price,offers,mean_Area
0,2020-07,Bemowo,2147.079295,227,42.821322
1,2020-07,Bialoleka,1972.745946,185,42.328973
2,2020-07,Bielany,2183.816901,284,41.346092
3,2020-07,Downtown,3138.840191,1677,41.331896
4,2020-07,Mokotow,2590.152330,1116,42.611604
...,...,...,...,...,...
123,2021-02,Wawer,2141.463415,41,40.964634
124,2021-02,Wilanow,2680.153153,222,46.356036
125,2021-02,Wlochy,2156.345238,84,38.609643
126,2021-02,Wola,2709.101102,1543,40.891419


In [12]:
df_monthly_dtr.to_csv("monthly_rent_by_district.csv", index=False)

In [13]:
df_monthly_dtr["monthly_offers"] = df_monthly_dtr.groupby('year_month')['offers'].transform(sum)

In [14]:
df_monthly_dtr["offers_share"] = df_monthly_dtr.offers  / df_monthly_dtr.monthly_offers *100

In [15]:
df_monthly_dtr.sort_values(by="year_month",inplace=True)
df_monthly_dtr.reset_index(inplace=True, drop=True)

In [16]:
df_monthly_dtr["change"]=df_monthly_dtr.groupby("district").total_monthly_price.pct_change()*100
df_monthly_dtr["change"]=df_monthly_dtr["change"].fillna(0)

In [17]:
top_5_districts = df_monthly_dtr.groupby("district", as_index=False).sum().sort_values(by="offers", ascending=False).district.unique()[:5]

In [18]:
data=[]
for dtr in top_5_districts :
    df_temp = df_monthly_dtr.query("district=='{}'".format(dtr))
    df_temp.reset_index(inplace=True, drop=True)
    
    trace=go.Scatter(
            x=df_temp.year_month,
            y=df_temp.total_monthly_price,
            hovertemplate= '%{text}',
            showlegend=False,
            mode='lines+markers',
            line=dict(color="grey", dash="dash"),
            marker=dict(
            color=df_temp.change,
            colorscale = custom_scale,
            cauto=False,
            cmin=-1,
            cmax=1,
            size=df_temp.offers_share,opacity=1,

                showscale=True,
                colorbar=dict(title="<b>monthly change<br> [%] <br>")
             
        
            
            ),
            name="",
    
        text=['''<b>District: {}<br>Mean monthly price: {:.0f}</b></br><br>Change from last month {:.2f}%<br>Offers count: {:.0f}<br>District share in all offers: {:.0f}%'''\
                              .format(df_temp.district[i],
                                      df_temp.total_monthly_price[i],
                                       df_temp.change[i],
                                      df_temp.offers[i],
                                         df_temp.offers_share[i],
                                     ) for i in range(0, df_temp.shape[0])],

        )
    
    
    data.append(trace)

# Adding text trace to show district names next to first point of the line graph
names_df = df_monthly_dtr[df_monthly_dtr.district.apply(lambda x: x in top_5_districts)].query("year_month=='2020-07'")
names_df.reset_index(inplace=True, drop=True)

dtr_names_trace = go.Scatter(
            x=names_df.year_month,
            y=names_df.total_monthly_price,
            text = names_df.district,
            mode="text+markers",
            textposition="top center",
            hoverinfo="skip",
            showlegend=False,
            marker=dict(
            
                size=names_df.offers_share,opacity=0)
            

        
            
            )

data.append(dtr_names_trace)
    
figure=go.Figure(
data=data,
layout=go.Layout(
        title="<b>Average monthly rent by District</b><br>line=rent, size=districts offers share , color=monthly change",
            yaxis=dict(title="<b>Rent [PLN]"),
        xaxis=dict(title="<b>Month",type="category")

    
    ))


iplot(figure)
    
    